<a href="https://colab.research.google.com/github/bennsamuel/Pneumonia-Project/blob/master/cnn_parameter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deletes log folder
should not be used. Was implemented when cnn algorithm was not finshed to mingle around.


In [0]:

# %cd /content/drive/My Drive/Group 77 - Pneumonia/logs
# !rm -d -r *


# START HERE

In [1]:
# START HERE

# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# After executing the cell above, Drive
# files will be present in "/content/drive/My Drive".
!ls "/content/drive/My Drive/Group 77 - Pneumonia/pickle/"

old  test_data.pickle  training_data.pickle  validation_data.pickle


In [3]:
IMG_SIZE = 224
NUM_CLASSES = 3

#                0          1         2
CATEGORIES = ["NORMAL","BACTERIA", "VIRUS"]


%cd /content/

/content


## `DATA SET which should be used for training is defined here.`

In [4]:
import pickle
import numpy as np
from keras.utils.np_utils import to_categorical
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import time






def parameter_study(WIDTH_SHIFT, HEIGHT_SHIFT, ZOOM, ROTATION, MIRROR_V):

    data_set_path = "/content/drive/My Drive/Group 77 - Pneumonia/pickle_augmented/aug-{}_{}_{}_{}_{}.pickle".format(WIDTH_SHIFT,HEIGHT_SHIFT,ZOOM,ROTATION,MIRROR_V)
    # extract name from dataset
    specific_data_set = data_set_path.split('/')[-1]

    # train set
    data = pickle.load(open(data_set_path, "rb"))

    # validation set
    test_data = pickle.load(open("/content/drive/My Drive/Group 77 - Pneumonia/pickle/validation_data.pickle", "rb"))

    print("Mounted data set: {}".format(specific_data_set))

    # splitting data in training data and validation data 
    # creating normalized SHUFFLED arrays
    # DUE TO REFACTORING validaiton set it called test set
    

    X = []
    y = []

    for features, label in data:
        X.append(features)
        y.append(label)
        
    X_train = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
    y_train = y

    X_test = []
    y_test = []

    for features, label in test_data:
        X_test.append(features)
        y_test.append(label)
        
    X_test = np.array(X_test).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

    X_train = np.asarray(X_train)
    y_train = np.asarray(y_train)

    # X_val = np.asarray(X_val)
    # y_val = np.asarray(y_val)

    X_test = np.asarray(X_test)
    y_test = np.asarray(y_test)

    X_train = X_train/255.0
    # X_val = X_val/255.0
    X_test = X_test/255.0



    y_train = to_categorical(y_train, NUM_CLASSES)
    y_test = to_categorical(y_test, NUM_CLASSES)


    y_train.shape

    NAME = "{data_set} - {time}".format(data_set = specific_data_set, time = time.strftime("%Y-%m-%d_%H:%M:%S"))

    tensorboard = TensorBoard(log_dir = "/content/drive/My Drive/Group 77 - Pneumonia/logs/{}".format(NAME))

    model = Sequential()

    model.add(Conv2D(64, kernel_size=(3, 3),
                    strides=2,
                    activation='relu',
                    input_shape=X_train.shape[1:]))
    model.add(Conv2D(64, kernel_size=(3, 3), strides=2, activation='relu'))
    # model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(128, kernel_size=(3, 3), strides=2, activation='relu'))
    model.add(Conv2D(128, kernel_size=(3, 3), strides=2, activation='relu'))
    # model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(256, kernel_size=(3, 3), strides=2, activation='relu'))
    model.add(Conv2D(256, kernel_size=(3, 3), strides=2, activation='relu'))
    # model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Flatten())

    # w/o regularization
    # model.add(Dense(128, activation='relu'))
    # model.add(Dropout(0.5))

    # model.add(Dense(128, activation='relu'))
    # model.add(Dropout(0.5))

    # with regularization
    model.add(Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l=0.1)))
    model.add(Dropout(0.5))

    model.add(Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l=0.1)))
    model.add(Dropout(0.5))

    # Dense 3 softmax because we have 3 classes
    model.add(Dense(NUM_CLASSES, activation='softmax'))

    print("Mounted data set: {}".format(specific_data_set))

    model.layers

    precision = tf.keras.metrics.Precision()
    recall = tf.keras.metrics.Recall()

    model.compile(loss="categorical_crossentropy", 
                  optimizer="adam", 
                  metrics=["accuracy", precision, recall])

    model.fit(X_train, y_train, batch_size=128, epochs=30, validation_data=(X_test,y_test), callbacks = [tensorboard])


Using TensorFlow backend.


In [0]:

for width_shift in range(0, 11, 5):
    for height_shift in range(0, 11, 5):
      for zoom in range(0, 22, 10):
        for rotation in range(0, 22, 10):
          for mirror_v in range(2):
            mir_v = False if mirror_v == 0 else True
            parameter_study(width_shift, height_shift, float(zoom)/100, rotation, mir_v)

Mounted data set: aug-0_0_0.0_0_False.pickle
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Mounted data set: aug-0_0_0.0_0_False.pickle
Train on 5331 samples, validate on 1570 samples
Epoch 1/30
5331/5331 [==============================] - 10s 2ms/sample - loss: 19.8682 - acc: 0.3361 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 8.7798 - val_acc: 0.2567 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/30
5331/5331 [==============================] - 7s 1ms/sample - loss: 5.2096 - acc: 0.4588 - precision: 0.6034 - recall: 0.1418 - val_loss: 2.7656 - val_acc: 0.6879 - val_precision: 0.7857 - val_recall: 0.5465
Epoch 3/30
5331/5331 [==============================] - 7s 1ms/sample - loss: 2.0008 - acc: 0.6269 - precision: 0.6772 - recall: 0.5183 - val_loss: 1.2897 - val_acc: 0.7452 - val_precision: 0.7811 - val_recall: 0.6955
Epoch 4/30
5331/5331 [==============================] - 7s 1ms/sample - loss: 1.0705 - acc: 0.6890 - precisio